In [6]:
from src.file_processor import chunk_text
import streamlit as st

import time
from langchain.chat_models import init_chat_model
from src.utils import init_llm
from src.prompts import prompt_template
from src.data_model import Data
from src.input import create_input_list

In [ ]:
!pip install streamlit

In [7]:
def chunk_text(file_path, lines_per_chunk=10):
    """
    Extracts text from a .txt file and divides it into chunks, each containing a specified number of lines.
    
    Args:
        file_path (str): The path to the text file.
        lines_per_chunk (int): The number of lines per chunk. Defaults to 10.
        
    Returns:
        list: A list of strings, where each string is a chunk containing up to 'lines_per_chunk' lines.
    """
    chunks = []
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            
        # Iterate over the lines in steps of 'lines_per_chunk'
        for i in range(0, len(lines), lines_per_chunk):
            # Join the lines to form a chunk, remove tab characters, and strip any trailing whitespace
            chunk = ''.join(lines[i:i+lines_per_chunk]).replace('\t', '').strip()
            if chunk:  # Only add non-empty chunks
                chunks.append(chunk)
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")
        
    return chunks

In [8]:
chunks = chunk_text("test_data/nvivo_city_goals_lead.txt")

In [9]:
chunks

['\ufeffIn a world of abundant ready-made entertainments, how do we foster creativity and imagination?\nNeed to see humility with those in power, transparency, and acknowledgement that \'we don\'t know\'.\n"Around climate I crave strong leadership" - it\'s not there. \nHow to make time for anything. How can we slow down, enable young people to slow down? Check space for new ideas. \nPeople from minority backgrounds need to be on boards where decisions are made: excluded, unable to take part. \nFor trust, people need to be less fearful - have time. (enable connection to the local... more secure living).\nWe have issues with the council trusting us to do things for the city\'.\nTrust: understand the city leadership as humans, break down the distance.\nCan we drip-feed info on the why and how of decision - making better.\nCities assemble for imagination and curiosity, not scrutiny... no commitment to it.',
 'Can we use empty buildings as new spaces for democratic decision-making.\nBig bus

In [10]:
llm = init_llm()

In [11]:
input_list = create_input_list(chunks)

/Users/fredericklewis/Documents/Projects/dml/taxonomy_generation/Langchain/src/input.py:94: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  tool_example_to_messages(text, [tool_call])


In [12]:
input_list

[{'text': '\ufeffIn a world of abundant ready-made entertainments, how do we foster creativity and imagination?\nNeed to see humility with those in power, transparency, and acknowledgement that \'we don\'t know\'.\n"Around climate I crave strong leadership" - it\'s not there. \nHow to make time for anything. How can we slow down, enable young people to slow down? Check space for new ideas. \nPeople from minority backgrounds need to be on boards where decisions are made: excluded, unable to take part. \nFor trust, people need to be less fearful - have time. (enable connection to the local... more secure living).\nWe have issues with the council trusting us to do things for the city\'.\nTrust: understand the city leadership as humans, break down the distance.\nCan we drip-feed info on the why and how of decision - making better.\nCities assemble for imagination and curiosity, not scrutiny... no commitment to it.',
  'examples': [HumanMessage(content='Can we use empty buildings as new spa

In [13]:
len(input_list)

60

In [14]:
for key, value in input_list[0].items():
    print(key)
    print(value)
    print('\n')

text
﻿In a world of abundant ready-made entertainments, how do we foster creativity and imagination?
Need to see humility with those in power, transparency, and acknowledgement that 'we don't know'.
"Around climate I crave strong leadership" - it's not there. 
How to make time for anything. How can we slow down, enable young people to slow down? Check space for new ideas. 
People from minority backgrounds need to be on boards where decisions are made: excluded, unable to take part. 
For trust, people need to be less fearful - have time. (enable connection to the local... more secure living).
We have issues with the council trusting us to do things for the city'.
Trust: understand the city leadership as humans, break down the distance.
Can we drip-feed info on the why and how of decision - making better.
Cities assemble for imagination and curiosity, not scrutiny... no commitment to it.


examples
[HumanMessage(content='Can we use empty buildings as new spaces for democratic decision-ma

In [15]:
input_list[0]

{'text': '\ufeffIn a world of abundant ready-made entertainments, how do we foster creativity and imagination?\nNeed to see humility with those in power, transparency, and acknowledgement that \'we don\'t know\'.\n"Around climate I crave strong leadership" - it\'s not there. \nHow to make time for anything. How can we slow down, enable young people to slow down? Check space for new ideas. \nPeople from minority backgrounds need to be on boards where decisions are made: excluded, unable to take part. \nFor trust, people need to be less fearful - have time. (enable connection to the local... more secure living).\nWe have issues with the council trusting us to do things for the city\'.\nTrust: understand the city leadership as humans, break down the distance.\nCan we drip-feed info on the why and how of decision - making better.\nCities assemble for imagination and curiosity, not scrutiny... no commitment to it.',
 'examples': [HumanMessage(content='Can we use empty buildings as new space

In [16]:
# 5. Configure the LangChain runnable.
runnable = prompt_template | llm.with_structured_output(
    schema=Data,
    method="function_calling",
    include_raw=False
)

In [17]:
responses = runnable.batch(input_list)

In [18]:
responses

[Data(themes=[Theme(theme='Innovation & Creativity', matching_quotes='In a world of abundant ready-made entertainments, how do we foster creativity and imagination?\nCheck space for new ideas.\nCities assemble for imagination and curiosity, not scrutiny... no commitment to it.'), Theme(theme='Trust & Relationships', matching_quotes="For trust, people need to be less fearful - have time. (enable connection to the local... more secure living).\nWe have issues with the council trusting us to do things for the city'.\nTrust: understand the city leadership as humans, break down the distance."), Theme(theme='Leadership & Governance', matching_quotes="Need to see humility with those in power, transparency, and acknowledgement that 'we don't know'.\n'Around climate I crave strong leadership' - it's not there."), Theme(theme='Inclusivity & Diversity', matching_quotes='People from minority backgrounds need to be on boards where decisions are made: excluded, unable to take part.'), Theme(theme='Y

In [19]:
from src.output import create_themes_dataframe

In [20]:
df = create_themes_dataframe(responses)

In [21]:
df

,Theme,Matching Quotes,Frequency
9,Community Engagement & Participation,Local area committees are good but not fully s...,96
44,Trust & Relationships,"For trust, people need to be less fearful - ha...",50
11,Community Identity & Pride,People living in Sheffield all their lives but...,42
31,Leadership & Governance,"Need to see humility with those in power, tran...",41
35,Public Services & Infrastructure,Migrant communities dumped in sink sites /forg...,32
28,Inclusivity & Diversity,People from minority backgrounds need to be on...,31
36,Public Spaces & Accessibility,S3 not disability friendly otherwise would hav...,30
40,Socioeconomic Inequality,Big businesses in Sheffield.... very few.\nFun...,29
8,Community Cohesion,No glue points where community assets are rich...,28
48,Youth Engagement & Future Focus,How to make time for anything. How can we slow...,28
